# Solving a semi-infinite program (SIP) in EAGO

[Matthew Wilhelm](https://psor.uconn.edu/person/matthew-wilhelm/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

[Robert Gottlieb](https://psor.uconn.edu/person/robert-gottlieb/)  
Department of Chemical and Biomolecular Engineering, University of Connecticut

### Using EAGO to solve a SIP

Semi-infinite programming remains an active area of research. In general, the solution of semi-infinite programs with nonconvex semi-infinite constraints of the following format are extremely challenging:

$
\begin{align}\label{form:SIPExplicit}
    f^*=&\min_{\mathbf x}f(\mathbf{x}) \\
    &{\rm s.t.}\; \max_{\mathbf{p}}{g(\mathbf{x,p})}\le 0 \\
    & \mathbf{x} \in X = \{x \in \mathbb{R} : \mathbf{x^{L}} \leq \mathbf{x} \leq \mathbf{x^{U}}\} \\
    & \mathbf{p} \in P = \{p \in \mathbb{R} : \mathbf{p^{L}} \leq \mathbf{p} \leq \mathbf{p^{U}}\} \\
\end{align}
$

EAGO implements the SIPRes algorithm of [1] to determine a globally optimal solution to problems of the above form. This accomplished using the `sip_solve` function which returns the optimal value, the solution, and a boolean feasibility value. The function `sip_solve` accepts decision variable bounds, uncertain variable bounds, the objective function, a vector of callback functions for the semi-infinite constraints, and then keyword arguments. Options for the SIPRes routine are prefixed with `res_sip_` such as `res_sip_absolute_tolerance`. To illustrate the function's use, a simple example is presented here which solves the following problem:

$
\begin{align}
    f(\mathbf{x}) &= (1/3)x_1^2 + x_2^2 + x_1/2 \\
    g(\mathbf{x},p) &= (1-(x_1^2)(p^2))^2 - x_1p^2 - x_2^2 + x_2 \leq 0 \\
     &{\; \qquad}\mathbf{x} \in X = [-1000, 1000]^2 \\
     &{\; \qquad}p \in P = [0.0, 1.0]  \label{ex:equalSIP}
\end{align}
$

In [1]:
using EAGO, JuMP

# Define semi-infinite program
f(x) = (1/3)*x[1]^2 + x[2]^2 + x[1]/2
gSIP(x,p) = (1.0 - (x[1]^2)*(p[1]^2))^2 - x[1]*p[1]^2 - x[2]^2 + x[2]

x_l = [-1000.0, -1000.0]
x_u = [1000.0, 1000.0]
p_l = [0.0]
p_u = [1.0]

# Create optimizer for use in solving SIP
factory = () -> EAGO.Optimizer(SubSolvers())
opt = optimizer_with_attributes(factory, "cut_max_iterations" => 0,
                                         "subgrad_tighten" => false,
                                         "absolute_tolerance" => 1E-4,
                                         "relative_tolerance" => 1E-4,
                                         "verbosity" => 0)
m = JuMP.Model(opt)
sip_result = sip_solve(SIPRes(), x_l, x_u, p_l, p_u, f, Vector{Any}([gSIP]), res_sip_absolute_tolerance = 1E-3)

println("")
println("The global minimum of the semi-infinite program is between: $(sip_result.lower_bound) and $(sip_result.upper_bound).")
println("The global minimum is attained at: x = $(sip_result.xsol).")
println("Is the problem feasible? $(sip_result.feasibility).")


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

-----------------------------------------------------------------------------------------------------------------------------
|  Iteration #  |     Nodes    | Lower Bound  |  Upper Bound  |      Gap     |     Ratio    |     Time     |    Time Left   |
-----------------------------------------------------------------------------------------------------------------------------
 
Empty Stack: Exhaustive Search Finished
First Solution Found at Node 1
LBD = -0.18849749906629681
UBD = -0.18748812500959877
Solution is :
    X[1] = -0.7499999861886596
    X[2] = 0.0
 
solved LowerProblem : -0.18748812500959877 [-0

## References
1. A. Mitsos, Global optimization of semi-infinite programs via restriction of the right-hand side, Optimization, 60:10-11, 1291-1308, [DOI:10.1080/02331934.2010.527970](DOI:10.1080/02331934.2010.527970)